In [1]:
# Import libraries and dependencies
import ccxt
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import talib as ta
import yfinance as yf 
import datetime as dt 

from CronJobs.Utility_Functions import Functions

# Downloading the data from yfinance

### Set the currencies, date range and the interval for which you want the data

In [16]:
currs_list = [ 'BTC-AUD', 'ETH-AUD', 'XRP-AUD' , 'LTC-AUD', 'ADA-AUD', 'XLM-AUD', 'BCH-AUD']     #
start_date = '2019-06-01'
end_date = '2021-09-01'
interval = '1h'

### Call yfinance api

In [17]:
df_data = yf.download(currs_list, start= start_date, end= end_date, interval= interval, group_by= 'ticker')
df_data.head(2)


### Split the data by currency and add it to the dictionary

Dictionary Structure: 
* Key= currency code 
* value = Dataframe conatining OHLCV data

In [18]:
df_data.shape

(17089, 42)

In [19]:
dict_ohlcv = {}

for curr in currs_list:

    df_ohlcv = df_data.loc[ :, ([curr]) ]
    curr_code = curr.replace('-', '/')

    df_ohlcv.columns = df_ohlcv.columns.droplevel(0)

    start_date = df_ohlcv.index[0].date().isoformat()
    end_date = df_ohlcv.index[-1].date().isoformat()
    num_records = (len(df_ohlcv))
    start_price = df_ohlcv.iloc[0]['Close']
    end_price = df_ohlcv.iloc[-1]['Close']

    print(f'Data summary for {curr_code}')
    print(f'    Start Date: {start_date}; End Date: {end_date}; NUmber of records: {num_records}')
    print(f'    Start Price: {start_price}; End Price: {end_price}')

    df_ohlcv.dropna(inplace=True)

    # Store the symbol name and history data in a dict 
    dict_ohlcv[curr_code] = df_ohlcv 

    print(f'Data for {curr_code} fetched and appended into the dictionary\n')


# Compute the Technical Indicators using

In [20]:
fast_window = 5
slow_window = 15

df_all_data = pd.DataFrame() 
for curr, data in dict_ohlcv.items():

    df = Functions.add_tech_indicators(data, fast_window, slow_window)
    
    df['Currency'] = curr 
    df['Returns'] = df['Close'].pct_change()

    df.dropna(inplace=True)    
    df_all_data = df_all_data.append(df)

df_all_data.shape

(117972, 18)

In [21]:
mask = df_all_data.index.date < pd.to_datetime('2021-07-01')
df_training = df_all_data.loc[mask].copy()
# df_training.tail()
df_training.to_csv('Resources/Training_data.csv', index=True)

## The next cells can be uncommented and run to test the strategy for month-wise data  

In [8]:
# mask = (df_all_data.index.date >= pd.to_datetime('2021-05-01')) & (df_all_data.index.date < pd.to_datetime('2021-09-01'))
# df_testing_data = df_all_data.loc[ mask ].copy()
# from_date = df_testing_data.index[0].date().isoformat()

# to_date = df_testing_data.index[-1].date().isoformat()

# print(f'Data filtered between {from_date} and {to_date}')

In [9]:
# df_testing_data.tail()

In [10]:
# df_testing_data.to_csv('Resources/Backtesting_July.csv')